In [ ]:
#Добавление дополнительных датасетов
df1=pd.read_csv('3rd_checkpoint/train.csv', sep=',')
df2=pd.read_csv('3rd_checkpoint/extra_train.csv', sep=',')
df3=pd.read_csv('2nd_checkpoint/extra_train.csv', sep=',')
df4=pd.read_csv('1st_checkpoint/extra_train.csv', sep=',')
df=pd.concat([df1, df2, df3, df4])
df

In [ ]:
#сохраняем по станциям
import seaborn as sns
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (20,3)

stations=df['station_id'].unique()
for station in stations:
    sub_df=df[df['station_id']==station]
    print(station)
    
    sub_df.to_csv('sub_datasets/train_'+str(station)+'.csv', index=False)

In [ ]:
#перезаписываем с восстановленным временным рядом
import datetime
import numpy as np

import os
folder_path='C:/Users/yulas/OneDrive/Документы/ITMO/NSS_lab/emergencyData/sub_datasets'
for file in os.listdir(folder_path):
    sub_df=pd.read_csv(os.path.join(folder_path, file))
    sub_df['date']=pd.to_datetime(sub_df['date'])
    sub_df.set_index('date', inplace=True)
    idx = pd.date_range(start='1/1/1985', end='31/10/2019', freq='D')
    
    
    
    new_df=pd.DataFrame()
    cols=['stage_avg', 'stage_min', 'stage_max', 'temp', 'water_code','ice_thickness', 'snow_height', 'place', 'discharge', 'year', 'month', 'day', 'delta_stage_max']
    for column in cols:
        stage_avg_df=sub_df[column]    
        stage_avg_df.index = pd.DatetimeIndex(stage_avg_df.index) 
        stage_avg_df = stage_avg_df.reindex(idx, fill_value=np.nan)
        new_df[column]=stage_avg_df
    print(new_df)
    new_df['station_id']=sub_df['station_id'][0]
    new_df.to_csv('sub_datasets_no_gaps/with_nan/no_gap_'+file, index=True)

In [ ]:
#заполняем пропуски федотом (для метеопараметров и временных рядов с недлительными пропусками (меньше полугода))
from fedot.core.chains.chain import Chain
from fedot.core.chains.node import PrimaryNode, SecondaryNode
from fedot.utilities.synth_dataset_generator import generate_synthetic_data
from fedot.utilities.ts_gapfilling import ModelGapFiller, SimpleGapFiller

folder_path='C:/Users/yulas/OneDrive/Документы/ITMO/NSS_lab/emergencyData/sub_datasets_no_gaps/with_nan'
output_folder_path='C:/Users/yulas/OneDrive/Документы/ITMO/NSS_lab/emergencyData/sub_datasets_no_gaps/no_gaps'

for file in os.listdir(folder_path):
    file_path = os.path.join(folder_path, file)

    new_df_open = pd.read_csv(os.path.join(folder_path, file))
    df = pd.read_csv(file_path)
    new_df = df
    df = df.fillna(9999)
    gap_names = ['stage_avg', 'stage_min', 'stage_max']
    for gap_name in gap_names:

        gap_data=df[gap_name]
        print(len(gap_data))

        # Filling in gaps using chain from FEDOT
        node_lagged = PrimaryNode('lagged')
        node_lagged.custom_params = {'window_size': 50}
        node_ridge = SecondaryNode('ridge', nodes_from=[node_lagged])
        ridge_chain = Chain(node_ridge)
        ridge_gapfiller = ModelGapFiller(gap_value=9999, chain=ridge_chain)


        without_gap_arr_ridge = ridge_gapfiller.forward_filling(gap_data)
        new_df[gap_name]=without_gap_arr_ridge

        #plt.plot(np.arange(len(gap_data)), gap_data)
        #plt.plot(np.arange(len(gap_data)), without_gap_arr_ridge)
        #plt.show()
        #print(len(gap_data))

    new_df.to_csv(os.path.join(output_folder_path, file), index=False )

In [ ]:
#заполняем пропуски для температуры
from statsmodels.tsa.seasonal import seasonal_decompose
folder_path='C:/Users/yulas/OneDrive/Документы/ITMO/NSS_lab/emergencyData/2nd_checkpoint/sub_datasets_no_gaps/no_gaps'
for file in os.listdir(folder_path):
    
    df=pd.read_csv(os.path.join(folder_path, file))
    df=df.rename(columns={"Unnamed: 0": "date"})
    df['date']=pd.to_datetime(df['date'])
   
    df[['temp2']] = df[['temp']].fillna(value=0)
        
    plt.rcParams["figure.figsize"] = (20,3)
    plt.plot(df['date'], df['temp'])
    plt.show()
    
    plt.rcParams["figure.figsize"] = (20,10)
    result = seasonal_decompose(df['temp2'], model='additive', period=365)
    result.plot()
    plt.show()
    
    plt.rcParams["figure.figsize"] = (20,3)
    seas=result.seasonal+result.trend.median()
    plt.plot(seas)
    plt.show()
    
    df['temp'][df['temp'].isna()]=seas
    plt.plot(df['temp'])
    plt.show()
    
    df=df.drop(['temp2'], axis=1)
    df.to_csv(os.path.join(folder_path, file), index=False)

In [ ]:
#заполняем пропуски для расходов
folder_path='C:/Users/yulas/OneDrive/Документы/ITMO/NSS_lab/emergencyData/3rd_checkpoint/sub_datasets_no_gaps/no_gaps'
for file in os.listdir(folder_path):
    if file in st_with_discharge:
        print(file)
        df=pd.read_csv(os.path.join(folder_path, file))
        df['date']=pd.to_datetime(df['date'])

        df[['discharge2']] = df[['discharge']].fillna(value=0)

        plt.rcParams["figure.figsize"] = (20,3)
        plt.plot(df['date'], df['discharge'])
        plt.show()

        plt.rcParams["figure.figsize"] = (20,10)
        result = seasonal_decompose(df['discharge2'], model='additive', period=365)
        #result.plot()
        #plt.show()

        plt.rcParams["figure.figsize"] = (20,3)
        seas=result.seasonal+result.trend.median()
        plt.plot(seas)
        plt.show()

        df['discharge'][df['discharge'].isna()]=seas
        plt.plot(df['discharge'])
        plt.show()

        df=df.drop(['discharge2'], axis=1)
        df.to_csv(os.path.join(folder_path, file), index=False)